In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
sns.set()

In [ ]:
fifa=pd.read_csv("../input/fifa-dataset/data.csv")
fifa.head()

In [ ]:
to_stay=["ID","Name","Age","Nationality","Club","Overall","Potential","Value","Wage","Real Face"]
fifa.drop(fifa.columns.difference(to_stay),axis="columns",inplace=True)

In [ ]:
fifa.head()

In [ ]:
fifa.set_index("ID",inplace=True)

In [ ]:
fifa.head()

In [ ]:
fifa.isnull().sum()

In [ ]:
fifa["Club"].dropna(axis="rows",inplace=True)

In [ ]:
fifa.dropna(axis="rows",inplace=True)

In [ ]:
fifa.isnull().sum()

In [ ]:
fifa.shape

In [ ]:
fifa.head(20)

Checking for outliers in Age.

In [ ]:
sns.boxplot(fifa["Age"])

In [ ]:
q1=np.percentile(fifa["Age"],25)
q3=np.percentile(fifa["Age"],75)
iqr=q3-q1
lower=q1-(1.5*iqr)
upper=q3+(1.5*iqr)
fifa["Age"][(fifa["Age"]<np.abs(lower))|(fifa["Age"]>upper)].max()

Hence, there are no outliers in Age.

In [ ]:
fifa.Value.unique

In [ ]:
fifa.info()

The goal here is to have only one Value column of their value in euros

In [ ]:
fifa["Value2"]=fifa["Value"].apply(lambda x: x.split("€")[1])


In [ ]:
fifa["Value3"]=fifa["Value2"].apply(lambda x:x.split("M")[0]*1000000 if x.split("M")==True else x.split("K")[0]*1000)
fifa

In [ ]:
fifa["Value3"]=fifa["Value2"].apply(lambda x: 1 if "M" in x else 0)
fifa["Value4"]=fifa["Value2"].apply(lambda x : x.split("M")[0] if "M" in x else x.split("K")[0]).astype(float)

In [ ]:
fifa["Value5"]=(fifa[fifa["Value3"]==1]["Value4"]*1000000)
a=fifa[fifa["Value3"]==0]
fifa["Value5"].fillna(a["Value4"]*1000,inplace=True)

In [ ]:
fifa

In [ ]:
fifa.drop(["Value4","Value3","Value2","Value"],axis="columns",inplace=True)

Rename value5 column to value

In [ ]:
fifa.rename(columns={"Value5":"Value"},inplace=True)


In [ ]:
fifa[fifa["Value"]==60000]

Change wage to value

In [ ]:
fifa["Wage"].unique()

In [ ]:
fifa["Wage2"]=fifa["Wage"].apply(lambda x: x.split("€")[1])
fifa["Wage3"]=fifa["Wage2"].apply(lambda x:x.split("K")[0]).astype(float)*1000
fifa.head()

In [ ]:
fifa.drop(["Wage","Wage2"],axis="columns",inplace=True)

In [ ]:
fifa.rename(columns={"Wage3":"Wage"},inplace=True)

In [ ]:
fifa

In [ ]:
fifa["Nationality"].value_counts()

In [ ]:
fifa["Nationality"].unique()

<h3>Countries in Africa</h3>
Senegal , Egypy,Gabon,Morocco,Algeria,Guinea,Ghana,Central African Rep., DR Congo,Ivory Coast,  Mali,Nigeria, Cameroon, Kenya,Cape Verde, Togo,Zimbabwe,Angola,Burkina Faso, Tunisia,Equatorial Guinea, Guinea Bissau,South Africa,Madagascar,Tanzania,Gambia,Benin,COngo,Mozambique,Sierra Leone,Zambia,CHad,Libya,Eritrea,Uganda,Niger,Mauritania,Namibia,Sudan,Ethiopia,Rwanda,South Sudan

### PEOPLE THAT THEIR REAL FACE WAS USED IN FIFA

In [ ]:
fifa["RF_Dummy"]=fifa["Real Face"].apply(lambda x: 1 if x=="Yes" else 0)

In [ ]:
fifa

We see below that the number of people that their real face was used in FIFA was 1654

In [ ]:
rf_used=np.sum(fifa["RF_Dummy"]==1)

In [ ]:
rf_notused=np.sum(fifa["RF_Dummy"]==0)

We see that  16264  people's real faces were not used 

In [ ]:
(rf_used/fifa.shape[0])*100

9.23% of people's real faces were used

In [ ]:
(rf_notused/fifa.shape[0])*100

We see a whooping percentage of 90.7 % people's real faces were not used

In [ ]:
plt.hist(fifa["Real Face"])

In [ ]:
sns.barplot(fifa["Real Face"],fifa["Overall"])

In [ ]:
fifa

In [ ]:
%matplotlib
sns.barplot(fifa["Overall"],fifa["RF_Dummy"])

From this graph , we see that there is a high likelihood of a player with overall rating of 79 and above for his real face to be used in the FIFA game.


In [ ]:
%matplotlib inline
sns.regplot(fifa["Overall"],fifa["Wage"])

In [ ]:
from scipy.stats import pearsonr
pearsonr(fifa["Overall"],fifa["Wage"])

In [ ]:
fifa

Overall does not really affect wage!!!!
<br>WOW!!

In [ ]:
sns.regplot(fifa["Overall"],fifa["Value"])

In [ ]:
pearsonr(fifa["Overall"],fifa["Value"])

Overall player rating does not have much correlation on a player's value too

In [ ]:
fifa.corr()

The only strong correlation in this dataset is between Player's Wage and Player's Value for sale and this is reasonable

In [ ]:
fifa[(fifa["Overall"]>60)&(fifa["Overall"]<80)]

### FOR AFRICAN PLAYERS

In [ ]:
africa=["Senegal" , "Egypyt","Gabon","Morocco","Algeria","Guinea","Ghana","Central African Rep.", "DR Congo","Ivory Coast",  "Mali","Nigeria", "Cameroon", "Kenya","Cape Verde", "Togo","Zimbabwe","Angola","Burkina Faso", "Tunisia","Equatorial Guinea", "Guinea Bissau","South Africa","Madagascar","Tanzania","Gambia","Benin","Congo","Mozambique","Sierra Leone","Zambia","Chad","Libya","Eritrea","Uganda","Niger","Mauritania","Namibia","Sudan","Ethiopia","Rwanda","South Sudan"]

In [ ]:
fifa_africa=fifa[fifa["Nationality"].isin(africa)]

In [ ]:
fifa_africa.corr()

For African players, Their overall fifa rating is strongly positively correlated with Value

In [ ]:
sns.scatterplot(fifa_africa["Overall"],fifa_africa["Value"])

In [ ]:
sns.regplot(fifa_africa["Overall"],fifa_africa["Value"])

Wage is also positively correlated with value of the player

In [ ]:
sns.regplot(fifa_africa["Wage"],fifa_africa["Value"])

# MACHINE LEARNING

Using Player features like Overall, Potential,Age to predict Player's Value and wage

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
model=LinearRegression()

In [ ]:
fifa.head()

In [ ]:
fifa=fifa.reset_index()

In [ ]:
X=fifa[["Age","Overall","Potential","Wage"]]

In [ ]:
y=fifa["Value"]

Using Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
models={"lr":{
    "model":LinearRegression()
},
       "svr":{
           "model":SVR()
       },
       "dtr":{
           "model":DecisionTreeRegressor()
       },
       "gbr":{
           "model":GradientBoostingRegressor()
       },
       "rtr":{
           "model":RandomForestRegressor()
       }}

In [ ]:
for name,model in models.items():
    pre=model["model"].fit(Xtrain,ytrain)
    print(f"The accuracy of {name} is {pre.score(Xtest,ytest)}")

Here, GradientBoostingRegressor is the best model

In [ ]:
gbr=GradientBoostingRegressor().fit(Xtrain,ytrain)
print(f"Your training accuracy is {gbr.score(Xtrain,ytrain)} while your test accuracy is {gbr.score(Xtest,ytest)}")

In [ ]:
ypred=gbr.predict(Xtest)

In [ ]:
gbr.predict([[20,76,78,50000]])

In [ ]:
mean_absolute_error(ytest,ypred)

In [ ]:
mean_squared_error(ytest,ypred)

In [ ]:
np.sqrt(mean_squared_error(ytest,ypred))

Thank you for viewing my project . Gracias...